In [45]:
# This script loops across slices and registers slice i-1 on slice i and 
# slice i+1 on slice i. Then, it applies the transformations on the neighbor 
# slices and outputs a 3d volume (x, y, 3) for each metric.

In [46]:
%matplotlib inline

import seaborn as sns
import pickle

import numpy as np
import scipy as sp
import nibabel as nib
import pandas as pd
import cv2
import os
import subprocess

from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
from PIL import Image
from scipy import misc
from subprocess import check_output

from sklearn import datasets
from sklearn.cluster import *
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
from sklearn.feature_extraction.image import grid_to_graph


from os import listdir
from os.path import isfile, join
from os import walk
from collections import OrderedDict
from collections import Counter

sns.set(font_scale=1.4)
sns.set_style("whitegrid", {'axes.grid' : False})

In [47]:
# Parameters
# FOLDER: folder that contains all slices and all metrics
Folder= "/Users/hanam/Documents/Tracts_testing_2/all_levels/"


# METRICS: list of metrics to register, e.g. [avf.nii.gz, ad.nii.gz]
Metrics= [
    "Axon_Density",
    "axon_equiv_diameter",
    "AVF_corrected",
    "GR_corrected",
    "Myelin_thickness",
    "MVF_corrected"
]

# METRIC_REF: file name of metric to use as reference for registration
#Metric_ref= nib.load(Folder/C1/Sample1_AD_nii.gz)


In [48]:
# Read FOLDER that contains all slices and all metrics
list_levels= next(walk(Folder))[1]
list_levels[0]

'C1'

In [49]:
len(list_levels)

31

In [50]:
#Create array of levels to have levels in proper order
array_levels= ["C1",
               "C2",
               "C3",
               "C4",
               "C5",
               "C6",
               "C7",
               "C8",
               "T1",
               "T2",
               "T3",
               "T4",
               "T5",
               "T6",
               "T7",
               "T8",
               "T9",
               "T10",
               "T11",
               "T12",
               "T13",
               "L1",
               "L2",
               "L3",
               "L4",
               "L5",
               "L6",
               "S1",
               "S2",
               "S3",
               "S4"
              ]

In [51]:
filelist = []
for folder_name in array_levels:
    folder_path = os.path.join(Folder, folder_name)
    filename = os.path.join(folder_path, "Volume4D_sym_cleaned.nii.gz")
    filelist.append(filename)

global filelist

# for folder_name in array_levels:
#     folder_path = os.path.join(Folder, folder_name)
#     print(folder_path)
#     filename = os.path.join(folder_path, "Volume4D_sym_cleaned.nii.gz")
#     print(filename)

In [52]:
for x in filelist:
    print(x)

/Users/hanam/Documents/Tracts_testing_2/all_levels/C1/Volume4D_sym_cleaned.nii.gz
/Users/hanam/Documents/Tracts_testing_2/all_levels/C2/Volume4D_sym_cleaned.nii.gz
/Users/hanam/Documents/Tracts_testing_2/all_levels/C3/Volume4D_sym_cleaned.nii.gz
/Users/hanam/Documents/Tracts_testing_2/all_levels/C4/Volume4D_sym_cleaned.nii.gz
/Users/hanam/Documents/Tracts_testing_2/all_levels/C5/Volume4D_sym_cleaned.nii.gz
/Users/hanam/Documents/Tracts_testing_2/all_levels/C6/Volume4D_sym_cleaned.nii.gz
/Users/hanam/Documents/Tracts_testing_2/all_levels/C7/Volume4D_sym_cleaned.nii.gz
/Users/hanam/Documents/Tracts_testing_2/all_levels/C8/Volume4D_sym_cleaned.nii.gz
/Users/hanam/Documents/Tracts_testing_2/all_levels/T1/Volume4D_sym_cleaned.nii.gz
/Users/hanam/Documents/Tracts_testing_2/all_levels/T2/Volume4D_sym_cleaned.nii.gz
/Users/hanam/Documents/Tracts_testing_2/all_levels/T3/Volume4D_sym_cleaned.nii.gz
/Users/hanam/Documents/Tracts_testing_2/all_levels/T4/Volume4D_sym_cleaned.nii.gz
/Users/hanam/Doc

In [53]:
nib.load(filelist[0])

In [54]:
x= nib.load(filelist[1])
print (x)

<class 'nibabel.nifti1.Nifti1Image'>
data shape (151, 151, 1, 6)
affine: 
[[0.05 0.   0.   0.05]
 [0.   0.05 0.   0.05]
 [0.   0.   1.   1.  ]
 [0.   0.   0.   1.  ]]
metadata:
<class 'nibabel.nifti1.Nifti1Header'> object, endian='<'
sizeof_hdr      : 348
data_type       : 
db_name         : 
extents         : 0
session_error   : 0
regular         : r
dim_info        : 0
dim             : [  4 151 151   1   6   1   1   1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
intent_code     : none
datatype        : float32
bitpix          : 32
slice_start     : 0
pixdim          : [1.   0.05 0.05 1.   1.   1.   1.   1.  ]
vox_offset      : 0.0
scl_slope       : nan
scl_inter       : nan
slice_end       : 0
slice_code      : unknown
xyzt_units      : 0
cal_max         : 0.0
cal_min         : 0.0
slice_duration  : 0.0
toffset         : 0.0
glmax           : 498
glmin           : 0
descrip         : 
aux_file        : none
qform_code      : unknown
sform_code      : scanner
qu

In [55]:
def preprocess_file(p,c,n):
    
    #1. Open each file
    x= nib.load(filelist[p])
    y= nib.load(filelist[c])
    z= nib.load(filelist[n])
    
    #2. Filter the metric you want
    p= x.get_data()
    p= p.transpose((1,0,2,3))
    p= p.squeeze(axis=2)
    p= p[...,0]
    
    c= y.get_data()
    c= c.transpose((1,0,2,3))
    c= c.squeeze(axis=2)
    c= c[...,0]
    
    n= z.get_data()
    n= n.transpose((1,0,2,3))
    n= n.squeeze(axis=2)
    n= n[...,0]
    
    #3. Create a new file for each file, name it "{}_pre.nii.gz".format(p)
    tmp_p= nib.Nifti1Image(p, x.affine, x.header)
    tmp_c= nib.Nifti1Image(c, y.affine, y.header)
    tmp_n= nib.Nifti1Image(n, z.affine, z.header)
    
    return "", "", ""

In [63]:
for i in range(1, len(filelist)-1):
    previous = i - 1
    current = i
    nex = i + 1
    
    tp, tc, tn = preprocess_file(previous,
                                 current,
                                 nex)
    
       
    #   register METRIC_REF(i-1) --> METRIC_REF(i)
    #     # outputs warp(i-1->1)

    
    
    try:
        subprocess.check_output(["isct_antsRegistration", "--dimensionality", "2", "--transform", "Affine[0.5]", 
            "--metric", "MeanSquares[tc, tp, 1, 5]",
            "--convergence", "100x100", "--shrink-factors", "8x4", "--smoothing-sigmas", "1x2vox",
            "--transform", "bsplinesyn[0.5,2]",
            "--metric", "MeanSquares[tc, tp, 1, 4]",
            "--convergence", "100x100x100x100", "--shrink-factors", "8x4x2x1", "--smoothing-sigmas", "0x0x0x0vox",
            "--output", "/Users/hanam/Documents/Tracts_testing_2/all_levels/warp_tp_tc",
            "--interpolation", "BSpline[3]", "--verbose", "1"])

    except subprocess.CalledProcessError, e:
         print "isct_antsRegistration stdout output:\n", e.output

    
    #   register METRIC_REF(i+1) --> METRIC_REF(i)
    #     # outputs warp(i+1->1)
    subprocess.check_output(["isct_antsRegistration", "--dimensionality", "2", "--transform", "Affine[0.5]",
            "--metric", "MeanSquares[tc, tn, 1, 5]",
            "--convergence", "100x100", "--shrink-factors", "8x4", "--smoothing-sigmas", "1x2vox",
            "--transform", "bsplinesyn[0.5,2]",
            "--metric", "MeanSquares[tc, tn, 1, 4]",
            "--convergence", "100x100x100x100", "--shrink-factors", "8x4x2x1", "--smoothing-sigmas", "0x0x0x0vox",
            "--output", "/Users/hanam/Documents/Tracts_testing_2/all_levels/warp_tp_tc",
            "--interpolation", "BSpline[3]", "--verbose", "1"])

    print(filelist[previous],
          filelist[current],
          filelist[nex])
    

isct_antsRegistration stdout output:
All_Command_lines_OK
Using double precision for computations.
  number of levels = 2
  number of levels = 4



CalledProcessError: Command '['isct_antsRegistration', '--dimensionality', '2', '--transform', 'Affine[0.5]', '--metric', 'MeanSquares[tc, tn, 1, 5]', '--convergence', '100x100', '--shrink-factors', '8x4', '--smoothing-sigmas', '1x2vox', '--transform', 'bsplinesyn[0.5,2]', '--metric', 'MeanSquares[tc, tn, 1, 4]', '--convergence', '100x100x100x100', '--shrink-factors', '8x4x2x1', '--smoothing-sigmas', '0x0x0x0vox', '--output', '/Users/hanam/Documents/Tracts_testing_2/all_levels/warp_tp_tc', '--interpolation', 'BSpline[3]', '--verbose', '1']' returned non-zero exit status -11

In [37]:
# Apply transfo across metrics
#   Loop across slices (i)
    for i in range(1, len(filelist)-1):

        # Loop across metrics (m)
        cd (list_levels[i-1])
        
    
#     apply warp(i-1->1) on METRICS[m]
#       output: METRICS[m, i-1]_r


#     apply warp(i+1->1) on METRICS[m]
#       output: METRICS[m, i+1]_r


#     concatenate: METRICS[m, i-1]_r, METRICS[m, i]_r, METRICS[m, i+1]_r
#       output: METRICS3D[m, i] --> 3D (x, y, 3) metric

SyntaxError: invalid syntax (<ipython-input-37-bd25957d298a>, line 3)